# LeNet 신경망
3-1. Simple Classifier에서 겪었던 문제점을 LeNet은 해결해 줄 수 있을까?<br>
한 번 제대로 시작해보자.

Reference:<br>
https://github.com/Steve-YJ/deep-learning-from-scratch-studying/blob/master/03_PyTorch%EB%A1%9C%2060%EB%B6%84%EB%A7%8C%EC%97%90%20%EB%94%A5%EB%9F%AC%EB%8B%9D%ED%95%98%EA%B8%B0/03.%20Neural%20Networks.ipynb

앞선 3-1. Simple Classifier의 데이터를 불러오자.<br>
대부분 학습 방법이 비슷하기에 <code>나만의 학습 패턴</code>을 만들어놓도록 하자

* Image Classifier 학습 패턴
    * #1. Import Library & Load Data
    * #2. Data Preprocessing
    * #3. Make Tensor
    * #4. 신경망 구성
    * #5. 모형 학습
    * #6. Accuracy Test
        참고<br>https://github.com/Steve-YJ/deep-learning-from-scratch-studying/blob/master/02_PyTorch_Introduction_Basic/Ch06_6.2_%EC%99%80%EC%9D%B8%20%EB%B6%84%EB%A5%98%EA%B8%B0%ED%95%98%EA%B8%B02_.ipynb

## #1. Import Library & Load Data

* PyTorch Library, scikit-learn Library, Pandas Library
* Load Data

In [1]:
# 기본 라이브러리 임포트
import os
import pickle
import numpy as np

# PyTorch 라이브러리 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

# scikit-learn 라이브러리 임포트
from sklearn.model_selection import train_test_split

# Pandas 라이브러리 임포트
import pandas as pd

### Load Data
Load pickle data

In [2]:
with open('../data.pkl', 'rb') as f:
    data = pickle.load(f)

In [3]:
# data

In [4]:
malimg = data

In [5]:
data = malimg[0]
target = malimg[1]
print(data)
print(target)

[[ 94.  32.   2. ... 129. 102. 124.]
 [ 54.  29.   6. ... 106. 119. 100.]
 [ 54.  29.   6. ... 120. 126. 104.]
 ...
 [ 65.  22.  59. ... 117. 116.  98.]
 [ 65.  22.  59. ... 117. 116.  98.]
 [ 65.  22.  59. ... 117. 116.  98.]]
[ 0.  0.  0. ... 24. 24. 24.]


In [6]:
print('shape of data: ', data.shape)
print('shape of target: ', target.shape)

shape of data:  (9339, 50176)
shape of target:  (9339,)


다음장 전처리에서 50176차원의 데이터를 224X224형상으로 변경해줘야한다!

## #2. Data Preprocessing
* 설명변수 목적변수 분할
* Train_Test Split

설명변수 목적변수 분할은 이미 되어있으므로 생략한다.

### Train_Test SPlit

In [7]:
# 데이터 집합을 훈련 데이터와 테스트 데이터로 분할해준다.
train_X, test_X, train_Y, test_Y = train_test_split(data, target, test_size = 0.2)
print(len(train_X))
print(len(test_X))

7471
1868


## #3. Make Tensor

완성된 train, test 데이터를 PyTorch의 Tensor 데이터로 변환해준다.<br>
즉 모델이 학습할 수 있는 데이터로 변환해주는 것이다.<br>
이 때 주의할 점은 두 가지이다.
* cuda()형태의 데이터로 변환
* shape reshape

In [8]:
train_X = torch.from_numpy(train_X).float().cuda()
train_Y = torch.from_numpy(train_Y).long().cuda()



RuntimeError: CUDA out of memory. Tried to allocate 358.00 MiB (GPU 0; 4.00 GiB total capacity; 1.40 GiB already allocated; 71.48 MiB free; 1.40 GiB reserved in total by PyTorch)

In [9]:
test_X = torch.from_numpy(test_X).float().cuda()
test_Y = torch.from_numpy(test_Y).long().cuda()

print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

RuntimeError: CUDA out of memory. Tried to allocate 358.00 MiB (GPU 0; 4.00 GiB total capacity; 1.40 GiB already allocated; 65.67 MiB free; 1.40 GiB reserved in total by PyTorch)

## #4. 신경망 구성

LeNet.py로 구성한 LeNet을 불러온다.
* 데이터의 사이즈가 224x224이기에 conv층을 3층 추가해주었다. 
* 기존 2Conv에서 5Conv로 변환

In [6]:
import LeNet

In [7]:
net = LeNet.Net()
net = net.cuda()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(32, 48, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(48, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=25, bias=True)
)


**해석해보자**

LeNet은 2개의 Conv Layer와 3개의 Linear Layer로 구성되어 있구나

모델의 학습 가능한 매개변수들은 <code>net.parameters()</code>에 의해 반환된다.

In [8]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

16
torch.Size([6, 1, 3, 3])


Q. torch.Size([6, 1, 3, 3])이란 무엇인가??

In [10]:
print(params[2].shape)

NameError: name 'params' is not defined

--Next--
* reshape
* 설명변수 목적변수 병합
* Mini-Batch 분할

데이터 shape 변환(reshape)

In [ ]:
data = data.reshape(9339, 1, 224, 224).shape

m

## #5. 모형 학습

이 신경망(LeNet)의 예상되는 입력의 크기는 32x32이다. 224x224 데이터를 학습시키기 위해서는 어떻게 해야할까?

이 전에 배우길, 32x32로 학습시키기 위해서는 이미지의 크기를 32x32로 변경해야 한다고 했다. 50176 크기의 데이터를 224x224로 변형해줘야 하지 않을까? 

In [10]:
data.shape

(9339, 50176)

In [11]:
data.reshape(9339, 1, 224, 224).shape

(9339, 1, 224, 224)

**Example**

In [12]:
input = torch.randn(1, 1, 224, 224)
input = input.cuda()
out = net(input)
print(out)

tensor([[ 0.0781,  0.0022, -0.0408, -0.0224,  0.1153,  0.0840, -0.0416,  0.0545,
         -0.0296,  0.0198,  0.0179,  0.0489, -0.0859, -0.1015, -0.0310,  0.0112,
          0.0644, -0.1010,  0.0435, -0.0988, -0.0015, -0.0356,  0.0389,  0.0066,
         -0.0210]], device='cuda:0', grad_fn=<AddmmBackward>)


size를 잘 맞춰줘야한다. 배웠지 Shape!

RuntimeError: size mismatch, m1: [1 x 46656], m2: [576 x 120] at C:/w/1/s/tmp_conda_3.7_100118/conda/conda-bld/pytorch_1579082551706/work/aten/src\THC/generic/THCTensorMathBlas.cu:290

In [15]:
net.zero_grad()
out.backward(torch.randn(1, 25).cuda())

In [16]:
print(out)

tensor([[ 0.0781,  0.0022, -0.0408, -0.0224,  0.1153,  0.0840, -0.0416,  0.0545,
         -0.0296,  0.0198,  0.0179,  0.0489, -0.0859, -0.1015, -0.0310,  0.0112,
          0.0644, -0.1010,  0.0435, -0.0988, -0.0015, -0.0356,  0.0389,  0.0066,
         -0.0210]], device='cuda:0', grad_fn=<AddmmBackward>)


지금까지 간단한 sample을 통해 두 가지를 배웠다.
* 첫 째, 신경망을 정의 하는 것
* 둘 째, 입력을 처리하고 backward를 호출하는 것

## #6. Accuracy Test

### #2. 손실 함수(Loss Function)

In [18]:
input.shape

torch.Size([1, 1, 224, 224])

In [19]:
out = net(input)
out

tensor([[ 0.0781,  0.0022, -0.0408, -0.0224,  0.1153,  0.0840, -0.0416,  0.0545,
         -0.0296,  0.0198,  0.0179,  0.0489, -0.0859, -0.1015, -0.0310,  0.0112,
          0.0644, -0.1010,  0.0435, -0.0988, -0.0015, -0.0356,  0.0389,  0.0066,
         -0.0210]], device='cuda:0', grad_fn=<AddmmBackward>)

In [21]:
# example of MSE
output = net(input)
target = torch.randn(25).cuda()
target = target.view(1, -1)

criterion = nn.MSELoss()
loss = criterion(output, target)
print('loss: ', loss)

loss:  tensor(1.3235, device='cuda:0', grad_fn=<MseLossBackward>)


### #3. 역전파(Backward)
오차(Error)를 역전파하기 위해서 <code>loss.backward()</code>만 해주면 된다.

In [23]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.], device='cuda:0')
conv1.bias.grad before backward
tensor([ 2.8392e-04,  1.5035e-03,  7.9345e-05,  3.2950e-04, -8.5058e-05,
        -5.7681e-04], device='cuda:0')


### #4. 가중치 갱신

In [24]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [25]:
import torch.optim as optim

# Optimizer를 생성한다.
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과정(training loop)에서는 다음과 같다.
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()